## Resources:
1. [Practiceal seq2seq](http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/)
2. [How to build a chatbot](https://github.com/suriyadeepan/practical_seq2seq)
    - his github code:https://github.com/Currie32/Chatbot-from-Movie-Dialogue/blob/master/Chatbot_Attention.ipynb
    

The conversations will be cleaned rather extensively to help the model to produce better responses. As part of the cleaning process, punctuation will be removed, rare words will be replaced with "UNK" (our "unknown" token), longer sentences will not be used, and all letters will be in the lowercase.

In [47]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import re # regular expression module
import time

In [2]:
# load the data
lines = open('./cornell/movie_lines.txt',encoding ='utf-8',errors = 'ignore').read().split('\n')
conv_lines = open('./cornell/movie_conversations.txt',encoding = 'utf-8',errors = 'ignore').read().split('\n')

- In **movies_line.txt** file,  it contains information about each movie character
- fields:
    - lines ID
    - character ID
    - movie ID
    - character name
    - text of the utterance

In [3]:
# the sentences that we will be using to train the model
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

- in ** movie_conversations.txt ** file, it contains the structure of the conversation
- fields:
    - characterID of the first character involved in the conversation
    - characterID of the second character involved in the conversation
    - movieID
    - list of the utterances that make the conversation, in chronological order

In [4]:
conv_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [18]:
# create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [19]:
# create a list of all of the conversations lines ids
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [20]:
convs[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [21]:
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [22]:
# Check if we have loaded the data correctly
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [34]:
print(len(questions))
print(len(answers))
print([questions[i] + answers[i] for i in range(10) ])

221616
221616
["Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.Well, I thought we'd start with pronunciation, if that's okay with you.", "Well, I thought we'd start with pronunciation, if that's okay with you.Not the hacking and gagging and spitting part.  Please.", "Not the hacking and gagging and spitting part.  Please.Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?", "You're asking me out.  That's so cute. What's your name again?Forget it.", "No, no, it's my fault -- we didn't have a proper introduction ---Cameron.", "Cameron.The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.", "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.Seems like she could get a date easy enough...", 'Why?Unsolved mystery.  She used to be really popu

In [35]:
def clean_text(text):
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text
    

In [41]:
# clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

print([clean_questions[i] + clean_answers[i] for i in range(10)])

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  againwell i thought we would start with pronunciation if that is okay with you', 'well i thought we would start with pronunciation if that is okay with younot the hacking and gagging and spitting part  please', 'not the hacking and gagging and spitting part  pleaseokay then how about we try out some french cuisine  saturday  night', 'you are asking me out  that is so cute that is your name againforget it', 'no no it is my fault  we did not have a proper introduction cameron', 'cameronthe thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does', 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she doesseems like she could get a date easy enough', 'whyunsolved mystery  she used to be really popular when she started high school then it was 

In [48]:
# find the length of the sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
    
print(lengths[:20])
# create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns = ['counts'])
lengths.describe()

[22, 14, 9, 14, 13, 1, 24, 1, 25, 9, 7, 8, 45, 9, 11, 1, 6, 7, 10, 2]


,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [49]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0
